In [1]:
import math
import random
import os
import imageio
import glob
import numpy as np
from PIL import Image
from datetime import timedelta, datetime
from IPython.display import display, clear_output

from keras.layers import Input, Dense, Conv2D, BatchNormalization, MaxPooling2D, UpSampling2D, Flatten, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.callbacks import TensorBoard


import cv2 #OpenCV library, used to make date stamps onto the images
import matplotlib.pyplot as plt
%matplotlib inline  

### About "Food-11" dataset
This is a dataset containing 16643 food images grouped in 11 major food categories. The 11 categories are Bread, Dairy product, Dessert, Egg, Fried food, Meat, Noodles/Pasta, Rice, Seafood, Soup, and Vegetable/Fruit. The whole dataset is divided in three parts: training, validation and evaluation.

In [2]:
# Load dataset
def load_dataset(path):
    files = os.listdir(path)
    x = []
    for file in files: # number of files to go through
        im = imageio.imread(os.path.join(path, file))
        x.append(im)
    x = np.asarray(x)
    return x

In [3]:
path_train_orig = './training'
path_val_orig = './validation'

x_train = load_dataset(path_train_orig)
x_val = load_dataset(path_val_orig)

print('original dataset:', np.shape(x_train), np.shape(x_val))

D:\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3080192 bytes but only got 0. Skipping tag 0
  warnings.warn(
D:\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2162688 bytes but only got 0. Skipping tag 0
  warnings.warn(
D:\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 0
  warnings.warn(
D:\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:792: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))
D:\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3473408 bytes but only got 0. Skipping tag 0
  warnings.warn(
D:\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:792: UserWarning: Corrupt EXIF data.  Expecting to read 12 byt

original dataset: (1111,) (1111,)


In [4]:
Foods = os.listdir(path_train_orig)
Foods.sort()
print(Foods)
labels = Foods

['0_0.jpg', '0_1.jpg', '0_10.jpg', '0_100.jpg', '0_11.jpg', '0_12.jpg', '0_13.jpg', '0_14.jpg', '0_15.jpg', '0_16.jpg', '0_17.jpg', '0_18.jpg', '0_19.jpg', '0_2.jpg', '0_20.jpg', '0_21.jpg', '0_22.jpg', '0_23.jpg', '0_24.jpg', '0_25.jpg', '0_26.jpg', '0_27.jpg', '0_28.jpg', '0_29.jpg', '0_3.jpg', '0_30.jpg', '0_31.jpg', '0_32.jpg', '0_33.jpg', '0_34.jpg', '0_35.jpg', '0_36.jpg', '0_37.jpg', '0_38.jpg', '0_39.jpg', '0_4.jpg', '0_40.jpg', '0_41.jpg', '0_42.jpg', '0_43.jpg', '0_44.jpg', '0_45.jpg', '0_46.jpg', '0_47.jpg', '0_48.jpg', '0_49.jpg', '0_5.jpg', '0_50.jpg', '0_51.jpg', '0_52.jpg', '0_53.jpg', '0_54.jpg', '0_55.jpg', '0_56.jpg', '0_57.jpg', '0_58.jpg', '0_59.jpg', '0_6.jpg', '0_60.jpg', '0_61.jpg', '0_62.jpg', '0_63.jpg', '0_64.jpg', '0_65.jpg', '0_66.jpg', '0_67.jpg', '0_68.jpg', '0_69.jpg', '0_7.jpg', '0_70.jpg', '0_71.jpg', '0_72.jpg', '0_73.jpg', '0_74.jpg', '0_75.jpg', '0_76.jpg', '0_77.jpg', '0_78.jpg', '0_79.jpg', '0_8.jpg', '0_80.jpg', '0_81.jpg', '0_82.jpg', '0_83.jpg',

The naming convention is {ClassID}_{ImageID}.jpg, where ID 0-10 refers to the 11 food categories respectively.

### Resize images

In [6]:
#Resize images
data_dir = "validation" # gives the path of folder you want to resize 
files = os.listdir(data_dir)
save_dir = data_dir + "_216"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
for file in files: # number of files to go through
    im = Image.open(os.path.join(data_dir, file))
    w, h = im.size
    
    if w > 255 and h > 255:
        
        # crop
        if w > h:
            w1 = (w - h)/2
            im = im.crop((w1, 0, w1 + h, h))
        elif h > w:
            h1 = (h - w)/2
            im = im.crop((0, h1, w, h1 + w))

        display(os.path.join(data_dir, file))
        im = im.resize((512,512), Image.ANTIALIAS)
        clear_output(wait=True)
        file_new = file.split('.jpg')[0] + '_s.jpg'
        im.save(os.path.join(save_dir, file_new), "JPEG")

'validation\\9_99.jpg'

In [ ]:
path_val_orig = './validation_216'
x_val, y_val = load_dataset(path_val_orig)

### Add date stamps

In [ ]:
# Make date stamps on images using OpenCV library
def put_date_on_image(img):
    random_time = timedelta(seconds=random.randint(0,math.pow(10,9)))
    random_date = (str(datetime.now() + random_time)[:10])
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size = cv2.getTextSize(random_date, font, 1, 2)
    x_max = len(img[0]) - text_size[0][0] # image length - text length
    y_min = text_size[0][1]  # text height
    y_max = len(img)  # image height
    pos = (random.randint(0, x_max), random.randint(y_min, y_max))
    draw = cv2.putText(img, random_date, pos, font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return draw.reshape(draw.shape[0], draw.shape[1], 3)

In [ ]:
# Show some of the images in jupyter
def show(imgs, num=10):
    n = num
    plt.figure(figsize=(20, 10))

    for i in range(n):
        ax = plt.subplot(2, n, i+1)
        plt.imshow(imgs[i])
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
# Process all food11 images
data_dir = "evaluation_" # gives the path
save = True # if you want to save to pc
save_dir = data_dir + "_datestamp"
if save and not os.path.exists(save_dir):
    os.mkdir(save_dir)
data_path = os.path.join(data_dir, '*g')
files = glob.glob(data_path)

data = []
for f in files:
    clear_output(wait=True)
    img = cv2.imread(f)
    draw = put_date_on_image(np.asarray(img))
    data.append(draw[:, :, ::-1])
    if not save:
        if '20' in f:
            break
    else:
        f_new = os.path.join(save_dir, f.split('\\')[1])
        display("saving images with date stamp:", f_new)
        cv2.imwrite(f_new, img)
clear_output(wait=True)
